# New Section

In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import csv

texts = []
labels_ = []

# Open the CSV file and read it line by line
with open("/content/complaints.csv", "r", encoding="utf-8") as file:
    # Create a CSV reader
    csv_reader = csv.reader(file)
    # Skip the header line
    next(csv_reader)
    # Iterate over each row in the CSV file
    for row in csv_reader:
      # Extract text and label from the row
      text = row[0]  # Remove leading and trailing quotes
      if len(row) > 1:
        label = row[1]  # Extract label from the second column
      else:
        label = None
      # Append text and label to the respective lists
      texts.append(text)
      labels_.append(label)


nltk.download("punkt")
# Preprocess the text data
texts = [word_tokenize(text.lower()) for text in texts]

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels_)

# Convert texts to sequences and pad them
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_sequence_length = max(len(seq) for seq in sequences)
data = pad_sequences(sequences, maxlen=max_sequence_length)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logistic_model = LogisticRegression(max_iter=200)
logistic_model.fit(X_train, y_train)
y_pred = logistic_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.6448569043812865


In [ ]:
y_pred = logistic_model.predict(X_test)
y_test_original = label_encoder.inverse_transform(y_test)
y_pred_original = label_encoder.inverse_transform(y_pred)

# Generate classification report
report = classification_report(y_test_original, y_pred_original)
# report = classification_report(y_test, y_pred_prob)
print(report)

                                      precision    recall  f1-score   support

   Attempts to collect debt not owed       0.41      0.02      0.05     14477
               Communication tactics       0.07      0.00      0.00      4160
                       Fraud or scam       0.06      0.01      0.01      2391
Incorrect information on your report       0.66      0.98      0.79     46122
          Struggling to pay mortgage       0.14      0.04      0.06      3537

                            accuracy                           0.64     70687
                           macro avg       0.27      0.21      0.18     70687
                        weighted avg       0.53      0.64      0.53     70687



In [ ]:

# Define model parameters
embedding_dim = 50
vocab_size = len(tokenizer.word_index) + 1

# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=6, batch_size=128, validation_split=0.2)


Epoch 1/8
4040/4040 [==============================] - 928s 229ms/step - loss: 0.5623 - accuracy: 0.7901 - val_loss: 0.3493 - val_accuracy: 0.8743
Epoch 2/8
4040/4040 [==============================] - 820s 203ms/step - loss: 0.3097 - accuracy: 0.8866 - val_loss: 0.3022 - val_accuracy: 0.8869
Epoch 3/8
4040/4040 [==============================] - 810s 201ms/step - loss: 0.2684 - accuracy: 0.9015 - val_loss: 0.2973 - val_accuracy: 0.8901
Epoch 4/8
4040/4040 [==============================] - 809s 200ms/step - loss: 0.2338 - accuracy: 0.9146 - val_loss: 0.2988 - val_accuracy: 0.8919
Epoch 5/8
 683/4040 [====>.........................] - ETA: 10:15 - loss: 0.1897 - accuracy: 0.9315

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

2209/2209 [==============================] - 158s 72ms/step - loss: 0.3135 - accuracy: 0.8912
Test Loss: 0.3134917616844177, Test Accuracy: 0.8912388682365417


In [ ]:
y_pred_prob = model.predict(X_test)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Convert encoded labels back to original labels
y_test_original = label_encoder.inverse_transform(y_test)
y_pred_original = label_encoder.inverse_transform(y_pred)

# Generate classification report
report = classification_report(y_test_original, y_pred_original)
print(report)

2209/2209 [==============================] - 157s 71ms/step
                                      precision    recall  f1-score   support

   Attempts to collect debt not owed       0.79      0.72      0.75     14477
               Communication tactics       0.78      0.82      0.80      4160
                       Fraud or scam       0.92      0.88      0.90      2391
Incorrect information on your report       0.92      0.95      0.94     46122
          Struggling to pay mortgage       0.91      0.94      0.93      3537

                            accuracy                           0.89     70687
                           macro avg       0.87      0.86      0.86     70687
                        weighted avg       0.89      0.89      0.89     70687



In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

NameError: name 'history' is not defined